<img src = "https://github.com/VeryFatBoy/notebooks/blob/main/common/images/img_github_singlestore-jupyter_featured_2.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">How to do Full-Text Search</h1>
    </div>
</div>

In [7]:
!pip cache purge

Files removed: 451


___

## Text Search With Java Lucene

- Text search V1 based on C Lucene
- Text search V2 added support for Java Lucene

Search V2 Features:

- BM25 scoring
- Fuzzy matching
- Proximity search
- Boosting
- Text search on JSON
- Customizable tokenizers (foreign language support and N-grams)

___

## BM25 Ranking

- **BM25 (Best Match 25)**: Ranking algorithm used in information retrieval that improves search relevance by considering term frequency, inverse document frequency, and document length normalization
- **TF-IDF (Term Frequency-Inverse Document Frequency)**: A statistical measure used in text analysis to evaluate how important a word is to a document within a collection, balancing its frequency in the document against how common it is across all documents

BM25 is:

- High-quality ranking function
- Used in Elastic, Solr, and OpenSearch
- Good heuristics for scoring with respect to term frequency and document length
- Better result quality compared to TF-IDF ranking

### Summary

- BM25 is a good, intuitive scoring algorithm to find most relevant documents
- Better than TF-IDF
- Applies "term saturation", e.g. a document that mentions elephants 200 times not twice as relevant as one that mentions elephants 100 times

In [9]:
%%sql
DROP TABLE IF EXISTS journals;

CREATE TABLE IF NOT EXISTS journals (
    volume VARCHAR(1000),
    name VARCHAR(1000),
    journal VARCHAR(1000),
    body LONGTEXT,
    KEY(volume),
    FULLTEXT USING VERSION 2 fts_index (journal, body)
);

++
||
++
++

In [15]:
%%sql
SHOW INDEXES FROM journals;

3 rows affected.

Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Index_options
journals,1,volume,1,volume,utf8_general_ci,None,None,None,YES,COLUMNSTORE HASH,,,
journals,1,fts_index,1,journal,utf8_general_ci,None,None,None,YES,FULLTEXT,,,
journals,1,fts_index,2,body,utf8_general_ci,None,None,None,YES,FULLTEXT,,,


In [16]:
%%sql
CREATE PIPELINE IF NOT EXISTS journal_pipeline AS
LOAD DATA S3 'zhou-fts/doc_1000.json'
CONFIG '{
   "region" : "us-west-1"}'
INTO TABLE journals
FORMAT JSON
( volume <- volume,
  name <- name,
  journal <- journal,
  body <- body
);

++
||
++
++

In [18]:
%%sql
TEST PIPELINE journal_pipeline LIMIT 1;

1 rows affected.

+--------+----------------------------------------------+--------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| volume |                     name                     |      journal       |                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [19]:
%%sql
START PIPELINE journal_pipeline;

++
||
++
++

In [20]:
%%sql
SHOW PIPELINES;

1 rows affected.

Pipelines_in_db_very_2c512,State,Scheduled
journal_pipeline,Running,False


In [21]:
%%sql
SELECT COUNT(*)
FROM journals;

1 rows affected.

COUNT(*)
1000


In [22]:
%%sql
SELECT journal, COUNT(*) AS frequency
FROM journals
GROUP BY journal
ORDER BY frequency DESC
LIMIT 10;

10 rows affected.

journal,frequency
Acta Crystallogr D Biol Crystallogr,410
Acta Crystallogr E Crystallogr Commun,329
3 Biotech,114
Acta Crystallogr D Struct Biol,35
Acta Crystallogr B Struct Sci Cryst Eng Mater,23
Acta Crystallogr A Found Adv,18
Acta Crystallogr A,14
Acta Biomater,12
Acta Crystallogr B,11
AAPS J,9


In [23]:
%%sql
STOP PIPELINE journal_pipeline;

++
||
++
++

In [24]:
%%sql
DROP PIPELINE journal_pipeline;

++
||
++
++

In [25]:
%%sql
OPTIMIZE TABLE journals FLUSH;

Table,Op,Msg_type,Msg_text


In [26]:
%%sql
SELECT journal
FROM journals
WHERE MATCH (TABLE journals) AGAINST ('body:optometry')
LIMIT 5;

3 rows affected.

journal
Acta Crystallogr E Crystallogr Commun
Acta Biomater
Acta Crystallogr E Crystallogr Commun


In [27]:
%%sql
SELECT journal
FROM journals
WHERE MATCH (TABLE journals) AGAINST ('body:dentistry OR body:teeth')
LIMIT 5;

5 rows affected.

journal
Acta Crystallogr D Biol Crystallogr
Acta Crystallogr D Biol Crystallogr
Acta Crystallogr D Biol Crystallogr
Acta Biomater
Acta Crystallogr D Biol Crystallogr


In [28]:
%%sql
SELECT journal
FROM journals
WHERE MATCH (TABLE journals) AGAINST ('body:dentist*')
LIMIT 5;

4 rows affected.

journal
Acta Crystallogr D Biol Crystallogr
Acta Crystallogr D Biol Crystallogr
Acta Biomater
Acta Crystallogr D Biol Crystallogr


In [29]:
%%sql
SELECT journal,
    BM25(journals, '(journal:"crystallogr" OR journal:"aging") AND (body:"teeth")') AS score
FROM journals
WHERE score
ORDER BY score DESC;

1 rows affected.

journal,score
Acta Crystallogr D Biol Crystallogr,3.021637201309204


___

## Fuzzy + Proximity Search

- **Fuzzy search**: Match words when a specified number of characters are different.

  Example: **brow~1** matches **brown**

- **Proximity search**: Find matching words within N words of each other.

  Example: **"brown quick fox"~3** matches **“quick brown fox”**

In [30]:
%%sql
DROP TABLE IF EXISTS sentences;

CREATE TABLE IF NOT EXISTS sentences (
    id INT,
    phrase VARCHAR(400),
    SORT KEY (id),
    SHARD KEY (id),
    FULLTEXT USING VERSION 2 KEY (phrase)
);

++
||
++
++

In [31]:
%%sql
INSERT sentences VALUES
(1, "The quick brown fox jumps over the lazy dog"),
(2, "Able was I ere I saw elba");

2 rows affected.

++
||
++
++

In [32]:
%%sql
OPTIMIZE TABLE sentences FLUSH;

Table,Op,Msg_type,Msg_text


In [33]:
%%sql
SELECT *
FROM sentences
WHERE MATCH (TABLE sentences) AGAINST ('phrase:brow~1');

1 rows affected.

id,phrase
1,The quick brown fox jumps over the lazy dog


In [34]:
%%sql
SELECT *
FROM sentences
WHERE MATCH (TABLE sentences) AGAINST ('phrase:"brown quick fox"~3');

1 rows affected.

id,phrase
1,The quick brown fox jumps over the lazy dog


___

## Foreign Languages

About 80 major languages and special tokenizers included with Java Lucene. Examples:

- Arabic
- Chinese
- French
- German
- Hebrew
- Japanese
- Korean
- Portuguese
- Turkish
- Whitespace (programming language)

### Why does this matter?  

Because if you use English for everything, it will:

1. Index common foreign words, such as 'le' and 'la' in French, and
2. Not correctly stem plurals, verb endings, etc. which means the index becomes bloated, slow and gives poor recall.

___

## Foreign Languages and Custom Analyzer Support

- Analyzer is code that:
  - Splits text into words (tokens)
  - Applies stemming (e.g. horses -> horse, running -> run)
  - Removes stop words (English: the, a ...; French: le, la, les ...)
- Analyzers are language-specific

## Special Analyzers

- Wikipedia
- Email
- Whitespace (does not break at hyphens, only at white space)
- N-grams

In [36]:
%%sql
DROP TABLE IF EXISTS french;

CREATE TABLE IF NOT EXISTS french (
    id INT,
    phrase VARCHAR(400),
    SORT KEY (id),
    SHARD KEY (id),
    FULLTEXT USING VERSION 2 KEY (phrase)
    INDEX_OPTIONS '{"analyzer" : "french"}'
);

++
||
++
++

In [37]:
%%sql
INSERT french VALUES
(1, "Ou et la wc?"),
(2, "Combien d'ans avez-vous?");

2 rows affected.

++
||
++
++

In [38]:
%%sql
OPTIMIZE TABLE french FLUSH;

Table,Op,Msg_type,Msg_text


In [39]:
%%sql
SELECT *
FROM french
WHERE MATCH (TABLE french) AGAINST("phrase:combien");

1 rows affected.

id,phrase
2,Combien d'ans avez-vous?


In [40]:
%%sql
DROP TABLE IF EXISTS french_withstopset;

CREATE TABLE IF NOT EXISTS french_withstopset (
    id INT,
    phrase VARCHAR(400),
    SORT KEY (id),
    SHARD KEY (id),
    FULLTEXT USING VERSION 2 KEY (phrase)
    INDEX_OPTIONS '{"analyzer" : {"french": {"stopset": ["stop", "stop1"]}}}'
);

-- Note: stopset overwrites default stop set, does not extend it.

++
||
++
++

In [41]:
%%sql
INSERT INTO french_withstopset VALUES
(1, "stop1 la folie");

1 rows affected.

++
||
++
++

In [43]:
%%sql
OPTIMIZE TABLE french_withstopset FLUSH;

Table,Op,Msg_type,Msg_text


In [44]:
%%sql
SELECT *
FROM french_withstopset
WHERE MATCH (TABLE french_withstopset) AGAINST ("phrase:stop1");

id,phrase


In [45]:
%%sql
SELECT *
FROM french_withstopset
WHERE MATCH (TABLE french_withstopset) AGAINST("phrase:la");

1 rows affected.

id,phrase
1,stop1 la folie


## Chinese, Japanese, Korean (CJK)

In [46]:
%%sql
DROP TABLE IF EXISTS chinese;

CREATE TABLE IF NOT EXISTS chinese (
    id INT,
    phrase VARCHAR(400),
    SORT KEY (id),
    FULLTEXT USING VERSION 2 KEY (phrase)
    INDEX_OPTIONS '{"analyzer" : "cjk"}'
);

++
||
++
++

In [47]:
%%sql
INSERT INTO chinese VALUES
(1, "百威啤酒是安海斯-布希公司產品之一 "),
(2, "中國青島啤酒集團和世界最大釀酒商美國安海斯-布希公司(Anheuser-Busch)達成協議，組成商 業策略聯盟。 ");

2 rows affected.

++
||
++
++

In [48]:
%%sql
OPTIMIZE TABLE chinese FLUSH;

Table,Op,Msg_type,Msg_text


In [49]:
%%sql
SELECT phrase,
    MATCH (TABLE chinese) AGAINST ("phrase:(威啤酒)") AS score
FROM chinese
WHERE score
ORDER BY score DESC;

2 rows affected.

phrase,score
百威啤酒是安海斯-布希公司產品之一,0.2615291476249695
中國青島啤酒集團和世界最大釀酒商美國安海斯-布希公司(Anheuser-Busch)達成協議，組成商 業策略聯盟。,0.13076457381248474


___

## N-grams

Index small chunks of words. For example, 'great' is indexed as 'gre', 'rea' and 'eat'.


### Why does this matter?  

Because you can now add more search features in your applications, such as auto-complete and tolerance for spelling errors. And it's fast because it uses the index.

- **Partial matching**: N-grams are great for autocomplete or type-ahead features allowing partial matches as users type, and help find relevant results even if a word is only partially typed or misspelled
- **Improved recall**: N-grams help in cases where you want to find matches with slight spelling variations or typos, as they break words into overlapping segments, which can improve recall, especially in applications where people might not know exact terms or phrases

In [50]:
%%sql
DROP TABLE IF EXISTS cities;

CREATE TABLE IF NOT EXISTS cities (
    id INT,
    name VARCHAR(400),
    SORT KEY (id),
    FULLTEXT USING VERSION 2 KEY (name)
    INDEX_OPTIONS '{"analyzer" : {"custom" : {"tokenizer" : "n_gram"}}}}'
);

++
||
++
++

In [51]:
%%sql
INSERT INTO cities VALUES
(1, "seattle"),
(2, "vancouver");

2 rows affected.

++
||
++
++

In [52]:
%%sql
OPTIMIZE TABLE cities FLUSH;

Table,Op,Msg_type,Msg_text


In [53]:
%%sql
SELECT id,
    SUBSTR (name, 1, 75),
    MATCH (TABLE cities) AGAINST ('name:seatle') AS score
FROM cities
WHERE score
ORDER BY score DESC;

-- Note: Seattle misspelled

2 rows affected.

id,"SUBSTR (name, 1, 75)",score
1,seattle,3.107238292694092
2,vancouver,0.2357606589794159


## Interesting Token Filters

- **snowball_porter**: Stemming for multiple languages (default tokenizers don't stem, e.g. *biking* does not match *bike*)
- **lower_case**: Converts all tokens to lower case
- **html_strip**: Strip html formatting
- **elision**: Removes elisions, e.g. "l'avion" (the plane) will be tokenized as "avion" (plane).

You can string together one tokenizer and multiple filters.

In [55]:
%%sql
DROP TABLE IF EXISTS a;

CREATE TABLE IF NOT EXISTS a (
    a INT,
    b VARCHAR(400),
    SORT KEY (a),
    FULLTEXT USING VERSION 2 KEY (b)
    INDEX_OPTIONS '{"analyzer" : {"custom" : {"tokenizer" : "standard", "token_filters": ["lower_case", "snowball_porter"]}}}'
);

++
||
++
++

In [56]:
%%sql
DROP TABLE IF EXISTS b;

CREATE TABLE IF NOT EXISTS b (
    a INT,
    b VARCHAR(400),
    SORT KEY (a),
    FULLTEXT USING VERSION 2 KEY (b)
    INDEX_OPTIONS '{"analyzer" : {"custom" : {"tokenizer" : "whitespace", "token_filters": ["elision", {"snowball_porter" : {"language": "French"}}]}}}'
);

++
||
++
++

___

## Text Search on JSON

- Full-text search now works on JSON
- Can search whole document or specific paths
- Can search for text that appears anywhere in document
